In [1]:
import imageio
from pathlib import Path
import matplotlib.pyplot as plt
import pandas
import pickle
import numpy
import os
from skimage.filters import gaussian
import sys
from tqdm import tqdm

repo_path = str(Path().resolve().parent.parent)
os.chdir(repo_path)
sys.path.append(repo_path)

from src.satellite_image_processing.normalize_index import (
    create_ndti_raster,
    create_ndwi_raster
)

In [3]:
image_class_path_list = list(filter(lambda path: "2022" in str(path), list(Path("data/bouctouche_class").iterdir())))

In [4]:
image_list = []
for image_class_path in image_class_path_list:
    with open(image_class_path, "rb") as f:
        image_list.append(pickle.load(f))

In [5]:
image_list_no_cloud = []
for image in image_list:
    if image.calculated_cloud_coverage < 0.01:
        image_list_no_cloud.append(image)

In [67]:
tides = pandas.read_csv("data/1805-01-MAY-2022_slev.csv", delimiter=",", skiprows=6, usecols=[0,1])
tides.columns = ["datetime", "height"]

In [86]:
import matplotlib.gridspec as gridspec

In [100]:
tides[3490:3510]

,datetime,height
3490,2022/09/23 17:00,1.23
3491,2022/09/23 18:00,1.24
3492,2022/09/23 19:00,1.37
3493,2022/09/23 20:00,1.51
3494,2022/09/23 21:00,1.74
3495,2022/09/23 22:00,1.90
3496,2022/09/23 23:00,2.17
3497,2022/09/24 00:00,2.29
3498,2022/09/24 01:00,2.49
3499,2022/09/24 02:00,2.71


In [93]:
date

'2022-09-25'

In [101]:
img_index_for_water_mask = 3
ndwi = create_ndwi_raster(image_list_no_cloud[img_index_for_water_mask].green_band, image_list_no_cloud[img_index_for_water_mask].nir_band)
water_mask = numpy.atleast_3d((ndwi > 0.01)*1)[:600, :1500, :]
rgb = image_list_no_cloud[img_index_for_water_mask].get_rgb_float_true_color_image()[:600, :1500, :]
for img_index in tqdm(range(len(image_list_no_cloud))):
    fig = plt.figure(figsize=(10, 5))
    gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1])
    axs = [plt.subplot(gs[0]), plt.subplot(gs[1])]
    date = image_list_no_cloud[img_index].date.split("T")[0]
    try:
        tide_height = tides[tides["datetime"] == date.replace("-", "/") + " 15:00"]["height"].values[0]
    except:
        tide_height = 0
    ndti = numpy.atleast_3d(create_ndti_raster(image_list_no_cloud[img_index].red_band, image_list_no_cloud[img_index].green_band))[:600, :1500]
    alpha_array = (numpy.ones_like(water_mask)*0.70*water_mask)[:600,:1500,0]
    axs[0].imshow(rgb)
    axs[0].imshow(ndti*water_mask, cmap="Reds",alpha=alpha_array, vmin=-0.15, vmax=0.35)
    axs[0].axis("off")
    axs[1].bar([0], [tide_height])
    axs[1].set_ylim(0, 3)
    axs[1].set_xticks([0], ["tide_height"])
    axs[1].set_xticks([0], ["tide_height"])
    axs[1].text(0, tide_height, str(tide_height), ha='center', va='bottom')
    axs[1].set_title("Tide Height")
    fig.suptitle(date)
    plt.tight_layout()
    plt.savefig(f"data/bouctouche_turbidity_img/{date}.png")
    plt.close()

  6%|▌         | 1/17 [00:00<00:10,  1.58it/s]C:\Users\ETE008168\OneDrive - INRS\Documents\turbidity_repo\turbidity-analysis\src\satellite_image_processing\normalize_index.py:11: RuntimeWarning: invalid value encountered in divide
  ndti = (red_band - green_band)/(red_band + green_band)
100%|██████████| 17/17 [00:09<00:00,  1.71it/s]


In [109]:
numpy.nanmax(ndti)

1.0

In [26]:
image_png_path_list = list(Path("data/bouctouche_turbidity_img").iterdir())

In [102]:
frames = []
for image_png_path in image_png_path_list:
    image = imageio.v2.imread(image_png_path)
    frames.append(image)

In [104]:
imageio.mimsave(
    'bouctouche.gif',
    frames,       
    duration=1000
)   

In [70]:
tides["height"].min()

0.2

In [71]:
tides["height"].max()

2.71